# Disaster Assist Scraping
- This script utilises Edge browser and Selenium, ensure you have web driver - https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/
- POC scrape data associated with this page: https://www.disasterassist.gov.au/find-a-disaster/australian-disasters and follow the link to the next page to get LGA names

The script would benefit from being more modular, however is POC currently and needs to be tested to ensure it works

The script also needs to add a pause when interactive with page to avoid issues

Bug in step 2

In [19]:
# Step 1: Import packages
from bs4 import BeautifulSoup
from datetime import datetime, timedelta, date
import getpass
import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
import time
import datetime




# Note you will need to chnage edge driver based on where you have it saved
edge_driver_path = r'C:\webdrivers\msedgedriver.exe'




## Interact wit dynamic page
The code below utilise Edge driver to extract URLs from Disaster Assist, this script is only concered with URLs from 2023 onwards currently

In [22]:
# Step 2

# Initialize the Edge driver
driver = webdriver.Edge(executable_path=edge_driver_path)

# URL of the website
base_url = 'https://www.disasterassist.gov.au/find-a-disaster/australian-disasters'

# Open the website
driver.get(base_url)

# Extract and print URLs for the year 2023
urls = []
while True:
    table_rows = driver.find_elements(By.CLASS_NAME, 'desk-result')
    for row in table_rows:
        # Extract start date and check if the year is 2023
        start_date = row.find_elements(By.TAG_NAME, 'td')[0].text
        if "2023" in start_date:
            urls.append(row.find_element(By.TAG_NAME, 'a').get_attribute('href'))
        else:
            # If the year is not 2023, break the loop
            break
    try:
        # Locate the next page button
        next_button = driver.find_element(By.ID, 'btnNext')
        
        # Scroll the button into view
        driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
        
        # Click the next button
        next_button.click()
    except:
        # If there is no next button, break the loop
        break

# Close the browser window
driver.quit()




['https://www.disasterassist.gov.au/Pages/disasters/current-disasters/New-South-Wales/bega-valley-bushfires-3-october-2023-onwards.aspx', 'https://www.disasterassist.gov.au/Pages/disasters/current-disasters/Victoria/south-east-victorian-storms-8-september-2023.aspx', 'https://www.disasterassist.gov.au/Pages/disasters/current-disasters/Northern-Territory/barkly-region-bushfire-september-2023.aspx', 'https://www.disasterassist.gov.au/Pages/disasters/current-disasters/New-South-Wales/north-eastern-nsw-bushfires-21-august-2023-onwards.aspx', 'https://www.disasterassist.gov.au/Pages/disasters/current-disasters/Victoria/north-victorian-storms-commencing-7-june-2023.aspx', 'https://www.disasterassist.gov.au/Pages/disasters/current-disasters/Queensland/southwest-queensland-flooding-4-June-15-June-2023.aspx', 'https://www.disasterassist.gov.au/Pages/disasters/current-disasters/Northern-Territory/east-arnhem-region-flooding-16-18-april-2023.aspx', 'https://www.disasterassist.gov.au/Pages/disaste

## Extract AGRN and LGA names of public anounced disasters
Looping through the URLs we are able to get AGRN and LGA list returned into pandas df

In [ ]:
# Step 3

data_list = []

for url in urls:
    # Get the current date and time
    extraction_date = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    # Extract relevant information from soup object
    # Modify the code below to extract the specific information you need from the URL pages
    agrn = soup.find('div', id='AGRN').text.strip()

    # Extract location names from <ul> and <li> elements
    location_container = soup.find('div', {'id': 'LGA'})
    # Extract location names from <ul> and <li> elements
    location_elements = soup.find('div', id='LGA').find_all('a')
    locations = ', '.join([location.text.strip() for location in location_elements])
    
    
    data_dict = {
        "agrn": agrn,
        'lga': locations,
        'extraction_date': extraction_date
    }
        
    
    # Append the dictionary to the data_list
    data_list.append(data_dict)

# Create a Pandas DataFrame from the list of dictionaries
df = pd.DataFrame(data_list)

# Print the DataFrame
df.to_csv("extract_poc_disaster_assist.csv", index=False) 
print(df)
